In [1]:
import cadquery as cq

from jupyter_cadquery import (
    versions,
    show, PartGroup, Part, 
    get_viewer, close_viewer, get_viewers, close_viewers, open_viewer, set_defaults, get_defaults, open_viewer,
    get_pick,
)

from jupyter_cadquery.replay import replay, enable_replay, reset_replay, disable_replay

enable_replay(False)

set_defaults(
    cad_width=780, 
    height=540, 
)

print()
versions()

Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay

jupyter_cadquery  3.1.0rc4
cad_viewer_widget 1.3.2
OCP               7.5.3


In [2]:
use_replay = True

if use_replay:
    enable_replay(False, False)
    reset_replay()
    show_object = replay
else:
    disable_replay()
    show_object = show


Enabling jupyter_cadquery replay


# Bar Mount

## Wall Bar Anchor

In [3]:
outer_height = 23
outer_depth = 20
inner_height = 22
inner_depth = 15

wall_thickness = 2
trench_depth = 9
trench_gap = 4.5

grip_thickness = 1.6
grip_mount_height = 6
grip_grip = 0.5
grip_length = 21

l_length = 60 - 2 * grip_length

bottom_thickness = 3

bottom_bracket_height = 7.2
bottom_bracket_indention = 6

In [4]:
lp00 = (0.,0)
lp01 = (inner_height,0.)
lp1 = (inner_height,wall_thickness)
lp2 = (bottom_thickness,wall_thickness)
lp3 = (bottom_thickness,inner_depth)
lp4 = (0,inner_depth)

grip_sketch = (
    cq.Sketch()
    .segment(lp00, lp01)
    .segment(lp1)
    .segment((inner_height - trench_depth,wall_thickness))
    .segment((inner_height - trench_depth,wall_thickness + trench_gap + grip_grip))
    .segment((inner_height,wall_thickness + trench_gap))
    .segment((inner_height,wall_thickness + trench_gap + grip_thickness))
    .segment((inner_height - trench_depth,wall_thickness + trench_gap + grip_thickness + grip_grip))
    .segment((inner_height - trench_depth - wall_thickness,wall_thickness + trench_gap + grip_thickness))
    .segment((grip_mount_height,wall_thickness))
    .segment(lp2)
    .segment((bottom_thickness,inner_depth - wall_thickness))
    .segment((bottom_bracket_height,inner_depth - bottom_bracket_indention))
    .segment((bottom_bracket_height,inner_depth - bottom_bracket_indention + wall_thickness))
    .segment(lp3)
    .segment(lp4)
    .close()
    .assemble(tag='grip_face')
    .edges('%LINE',tag='grip_face')
    # .vertices('>X or >Y or <X or <Y')
    # .vertices()
    # .fillet(0.75)
    .reset()
)

show_object(grip_sketch)

Use the multi select box below to select one or more steps you want to examine


In [5]:
l_sketch = (
    cq.Sketch()
    .segment(lp00, lp01)
    .segment(lp1)
    .segment(lp2)
    .segment(lp3)
    .segment(lp4)
    .close()
    .assemble(tag='l_face')
    .edges('%LINE',tag='l_face')
    # .vertices()
    # .fillet(0.75)
    .reset()
)

show_object(l_sketch)

Use the multi select box below to select one or more steps you want to examine


In [6]:
def make_grip_section():
    grip_section = (
        cq.Workplane('ZY')
        .placeSketch(grip_sketch)
        .extrude(grip_length)
    )
    grip_section.faces('>X').tag('left_plane')
    grip_section.vertices('>X and <Y and <Z').tag('left_point')
    grip_section.vertices('>X and <Y and >Z').tag('left_point_up')
    grip_section.faces('<X').tag('right_plane')
    grip_section.vertices('<X and <Y and <Z').tag('right_point')
    grip_section.vertices('<X and <Y and >Z').tag('right_point_up')
    return grip_section
    
show_object(make_grip_section())

Use the multi select box below to select one or more steps you want to examine


In [7]:
def make_l_section(length):
    inner_bore_height = 14

    l_section = (
        cq.Workplane('ZY')
        .placeSketch(l_sketch)
        .extrude(length)
        .faces(">Y[-2]").workplane()
        .center(l_length / 2, inner_bore_height)
        .cskHole(4, 8, 100, depth=None)        
    )
    l_section.faces('>X').tag('left_plane')
    l_section.vertices('>X and <Y and <Z').tag('left_point')
    l_section.vertices('>X and <Y and >Z').tag('left_point_up')
    l_section.faces('<X').tag('right_plane')
    l_section.vertices('<X and <Y and <Z').tag('right_point')
    l_section.vertices('<X and <Y and >Z').tag('right_point_up')
    return l_section

show_object(make_l_section(l_length))

Use the multi select box below to select one or more steps you want to examine


## Assamble the parts

In [8]:
disable_replay()
show_object = show

Removing replay from cadquery.Workplane (will show a final RuntimeWarning if not suppressed)


In [30]:
L = lambda *args: cq.Location(cq.Vector(*args))
C = lambda *args: cq.Color(*args)

def make_basic_mount():
    basic_mount = (
        cq.Assembly()
        .add(make_grip_section(), name="g0")
        .add(make_l_section(l_length), name="l0")
        .add(make_grip_section(), name="g1")
    )
    (
        basic_mount
        .constrain("g0?left_point", "Fixed")
        .constrain("g0?right_plane", "l0?left_plane", "PointInPlane")
        .constrain("g0?right_point", "l0?left_point", "Point")
        .constrain("g0?right_point_up", "l0?left_point_up", "Point")
        .constrain("l0?right_plane", "g1?left_plane", "PointInPlane")
        .constrain("l0?right_point", "g1?left_point", "Point")
        .constrain("l0?right_point_up", "g1?left_point_up", "Point")
    )
    basic_mount.solve()
    return basic_mount

show_object(make_basic_mount())

100% ⋮————————————————————————————————————————————————————————————⋮ (3/3)  0.03s


CadViewerWidget(anchor=None, cad_width=780, glass=False, height=540, pinning=False, theme='light', title=None,…

In [41]:
def make_mount():
    mount = (
        cq.Assembly()
        .add(make_basic_mount(), name="m0")
        .add(make_basic_mount(), name="m1")
    )
    (
        mount
        # .constrain("m0@g0?left_point", "Fixed")
        # .constrain("m0@g0?right_plane", "m1@l0?left_plane", "PointInPlane")
        # .constrain("m0@g0?right_point", "m1@l0?left_point", "Point")
        # .constrain("m0@g0?right_point_up", "m1@l0?left_point_up", "Point")
    )
    # mount.solve()
    return mount

show_object(make_mount())

ValueError: At least one constraint required